In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from Bio.SeqIO import parse
from tqdm import tqdm

In [3]:
def get_msa_info(msa_path):
    
    opn_msa = parse(msa_path, 'fasta')
    consensus_possitions = {}

    for line in tqdm(opn_msa):
        
        shift = 0
        
        for possition in range(len(line.seq)):
            
            if possition not in consensus_possitions:
    
                consensus_possitions[possition] = []
            consensus_possitions[possition].append(line.seq[possition])
            
            if line.seq[possition] == '-':
                shift += 1
                continue
    
    return consensus_possitions

def most_common(lst):

    return max(set(lst), key=lst.count)
    
def get_consensus(consensus_possitions):
    
    protoconsensus = ""
    
    for i in tqdm(consensus_possitions.keys()):
        if consensus_possitions[i].count('-') > len(consensus_possitions[i])/2:
        
            letter_possition = '-'
        else:
            consensus_possitions[i] = list(np.array(consensus_possitions[i])[np.array(consensus_possitions[i]) != '-'])
            
        letter_possition = most_common(consensus_possitions[i])
        
        protoconsensus += letter_possition
     
    protoconsensus = protoconsensus.replace('-', '').upper()
    
    return protoconsensus

In [4]:
opn_fasta = parse('./data/Tdt-proteins.fasta', 'fasta')
lens = []
seqids = []
usereads = []

for seq in opn_fasta:
    if 'DNA nucleotidylexotransferase' not in seq.description:
        continue
    if 'fragment' in seq.description.lower():
        continue
    if 'domain' in seq.description.lower():
        continue
    if 'Putative ' in seq.description.lower():
        continue
    if 'subunit' in seq.description.lower():
        continue
    lens.append(len(seq.seq))
    seqids.append(seq.id)

lens = np.array(lens)
seqids = np.array(seqids)

In [ ]:
right_part= lens[lens > np.median(lens)]
left_part= lens[lens < np.median(lens)]
right_edged = np.sort(right_part)[: int(len(right_part)/100*30)]
left_edged = np.sort(left_part)[int(len(left_part)/100*30) :]
rght_lim = np.max(right_edged)
left_lim = np.min(left_edged)

lenslim_subset_R = lens[lens < rght_lim]
ids_lims_R = seqids[lens < rght_lim]
filterd_seq = ids_lims_R[lenslim_subset_R > left_lim]

opn_fasta = parse('./data/Tdt-proteins.fasta', 'fasta')

with open('./data/Tdt-proteins_cleared_data.fasta', 'w') as new_fasta:
    for seq in opn_fasta:
        
        if seq.id not in filterd_seq:
            continue
            
        new_fasta.write(seq.format('fasta'))

In [ ]:
!mafft ./data/Tdt-proteins_cleared_data.fasta > ./data/Tdt-proteins_cleared_data.afa

## All Archaea

In [5]:
consensus_possitions = get_msa_info(f'./data/Tdt-proteins_cleared_data.fasta')
consensus = get_consensus(consensus_possitions)
print(consensus)

373it [00:00, 661.21it/s]
100%|██████████| 508/508 [00:00<00:00, 3203.76it/s]

MDRIRAPAVISQRKRQKGMHSPKLSCSYEIKFSKFVIFIMQRKMGMTRRTFLMELARRKGFRVESELSDSVTHIVAENNSYLEVLDWLKGQAVGDSSRFELLDISWFTACMEAGRPVDSEMKYRLMEQDQSPPLNTPESEVPSFIATKVSQYSCQRKTTLNNYNKKFTDAFEIMAENYEFKENEIFCLEFLRAASLLKSLPFPVTRMKDIQGLPCMGDQVRDIIEEIIEEGESSRVKEVLNDERYKSFKQFTSVFGVGVKTSEKWYRMGLRTVEEVKADKTLKLSKMQKAGFLYYEDLVSCVSKAEADAVSLIVKNTVCTFLPDALVTITGGFRRGKKIGHDIDFLITNPGPREDDELLHKGLLLYCDIIESTFVKEQLPSRKVDAMDHFQKCFAILKLYQPRVDNSSYNTSKKLDMAEVKDWKAIRVDLVITPFEQYAYALLGWTGSRQFGRDLRRYATHERKMILDNHALYDRRKRIFLKAGSEEEIFAHLGLDYVEPWERNAANA


## All sequenses

In [6]:
consensus_possitions = get_msa_info('./data/Tdt-proteins_cleared_data.fasta')

373it [00:00, 668.36it/s]


In [7]:
consensus = get_consensus(consensus_possitions)

100%|██████████| 508/508 [00:00<00:00, 2501.01it/s]


In [8]:
consensus

'MDRIRAPAVISQRKRQKGMHSPKLSCSYEIKFSKFVIFIMQRKMGMTRRTFLMELARRKGFRVESELSDSVTHIVAENNSYLEVLDWLKGQAVGDSSRFELLDISWFTACMEAGRPVDSEMKYRLMEQDQSPPLNTPESEVPSFIATKVSQYSCQRKTTLNNYNKKFTDAFEIMAENYEFKENEIFCLEFLRAASLLKSLPFPVTRMKDIQGLPCMGDQVRDIIEEIIEEGESSRVKEVLNDERYKSFKQFTSVFGVGVKTSEKWYRMGLRTVEEVKADKTLKLSKMQKAGFLYYEDLVSCVSKAEADAVSLIVKNTVCTFLPDALVTITGGFRRGKKIGHDIDFLITNPGPREDDELLHKGLLLYCDIIESTFVKEQLPSRKVDAMDHFQKCFAILKLYQPRVDNSSYNTSKKLDMAEVKDWKAIRVDLVITPFEQYAYALLGWTGSRQFGRDLRRYATHERKMILDNHALYDRRKRIFLKAGSEEEIFAHLGLDYVEPWERNAANA'

## Filtered data

In [9]:
consensus_possitions = get_msa_info('cleared_data.afa')

FileNotFoundError: [Errno 2] No such file or directory: 'cleared_data.afa'

In [ ]:
consensus = get_consensus(consensus_possitions)

## Draw tree

In [ ]:
from pandas import read_csv
sequence_annotation = read_csv('DNA_poly_betta_acrhea/uniprotkb_dna_polymerase_beta_AND_taxon_2023_09_11.tsv', sep='\t', index_col=0)
sequence_annotation['Taxonomic lineage']['A0A060HHM4'].split(', ')[-2]

In [ ]:
'''5.2.1 Psychrophiles and their sources
The optimum growth temperature required for psychrophiles to grow and reproduce is below 15°C whereas for psychrotrophs it ranges from 20°C to 25°C. Psychrophilic microorganisms inhabits in the cold regions of the Earth including polar zones, high mountains, glaciers, and deep oceans. Psychrophiles are also well accumulated in pockets of sea ice with high pH, salinity, inorganic nutrients, dissolved gases and light [20,28]. Psychrophilic microorganisms, including bacteria (e.g., Pseudoalteromonas and Pseudomonas), archaea (e.g., Methanogenium and Methanococcoides),'''

In [ ]:
'The most thermophilic microorganisms genera present in archaea domain Thermoplasma, Sulfolobus, Methanogens, Pyrococcus, and Pyrolobus (Rothschild and Mancinelli, 2001; Gadd, 2010; Sar et al., 2013).'

In [ ]:
import random 

def get_color(obj_dict):
    
    color = ''
    
    while color not in obj_dict.values() and color == '':
        
        color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    
    return color
    
taxonomy_color = {}
colors = {}
taxonomy_color['Unknown'] = 'black'

for line in parse('DNA_poly_betta_acrhea/uniprotkb_dna_polymerase_beta_AND_taxon_2023_09_08.fasta', 'fasta'):
    try:
        
        tax = sequence_annotation['Taxonomic lineage'][line.id.split('|')[1]].split(', ')[6]
    
    except:
        
        tax = 'Unknown'
    if 'Therm' not in tax:
        taxonomy_color[tax] = 'black'
        
    else:      
        if tax not in taxonomy_color: 
        
            taxonomy_color[tax] = get_color(taxonomy_color)
    
    colors[line.id] = taxonomy_color[tax]
    

In [ ]:
termos = []
psychros = []
most_temoph = ['Thermoplasma', 'Sulfolobus', 'Methanogens', 'Pyrococcus', 'Pyrolobus']
most_psychro = ['Methanogenium', 'Methanococcoides']

for term in most_temoph:
    
    termos.extend(list(sequence_annotation[sequence_annotation['Taxonomic lineage'].str.contains(term)].index))

for psy in most_psychro:

    psychros.extend(list(sequence_annotation[sequence_annotation['Taxonomic lineage'].str.contains(psy)].index))

    
lens = []
seqs = []
lines = []
taxes = []
ids = []

with open('DNA_poly_term.fasta', 'w') as opn_new_fasta:
    
    for line in parse('DNA_poly_betta_acrhea/uniprotkb_dna_polymerase_beta_AND_taxon_2023_09_08.fasta', 'fasta'):
        if line.id.split('|')[1] not in termos:
            continue
        if 'DNA polymerase beta' not in line.description:
            continue
        if 'fragment' in line.description.lower():
            continue
        if 'domain' in line.description.lower():
            continue
        if 'Putative ' in line.description.lower():
            continue
        if 'subunit' in line.description.lower():
            continue
        if line.seq in seqs:
            continue
        if ' '.join(line.description.split(' OX')[0].split('OS=')[1].split()[: 2]) in taxes:
            continue
        print(line.description)
        opn_new_fasta.write(line.format('fasta'))
        lens.append(len(line.seq))
        lines.append(line)
        taxes.append(' '.join(line.description.split(' OX')[0].split('OS=')[1].split()[: 2]))
        ids.append(line.id)
        seqs.append(line.seq)
        
        

In [ ]:
lens = []
seqs = []
lines = []
taxes = []
ids = []

with open('DNA_poly_psychros.fasta', 'w') as opn_new_fasta:
    
    for line in parse('DNA_poly_betta_acrhea/uniprotkb_dna_polymerase_beta_AND_taxon_2023_09_08.fasta', 'fasta'):
        if line.id.split('|')[1] not in psychros:
            continue
        if 'DNA polymerase beta' not in line.description:
            continue
        if 'fragment' in line.description.lower():
            continue
        if 'domain' in line.description.lower():
            continue
        if 'Putative ' in line.description.lower():
            continue
        if 'subunit' in line.description.lower():
            continue
        if line.seq in seqs:
            continue
        if ' '.join(line.description.split(' OX')[0].split('OS=')[1].split()[: 2]) in taxes:
            continue
        print(line.description)
        opn_new_fasta.write(line.format('fasta'))
        lens.append(len(line.seq))
        lines.append(line)
        taxes.append(' '.join(line.description.split(' OX')[0].split('OS=')[1].split()[: 2]))
        ids.append(line.id)
        seqs.append(line.seq)

## Thermophilus consensus

In [ ]:
consensus_possitions = get_msa_info(f'DNA_poly_term.afa')
consensus = get_consensus(consensus_possitions)
print(consensus)

In [ ]:
from Bio import Phylo
import matplotlib.pyplot as plt
import matplotlib 
import seaborn as sns

matplotlib.rc('font', size=5)
tree = Phylo.read("DNA_poly_betta_acrhea/uniprotkb_dna_polymerase_beta_AND_taxon_2023_09_08.newick", "newick")
tree.ladderize()  # Flip branches so deeper clades are displayed at top
fig, ax= plt.subplots(figsize=(100, 100))
Phylo.draw(tree,
           axes=ax,
           do_show=False,
           label_colors=colors)

plt.text(5, 7, 'Species:', fontsize=20)      
c = 20

for i in taxonomy_color.keys():
    
    plt.text(5, c, i, fontsize=20, color=taxonomy_color[i])
    c += 10

plt.axis('off')
#plt.savefig('KS_T2.pdf', bbox_inches='tight')
plt.show()

## TDT polymerase

In [ ]:
taxes = []
with open(f'/mnt/iscsidisk1/runs/runs-krivonos/PROJECTS/CONSENSUS_SEQ/TDT_bird/filtered.fasta', 'w') as opn_fasta:
    for seq in parse(f'/mnt/iscsidisk1/runs/runs-krivonos/PROJECTS/CONSENSUS_SEQ/TDT_bird/uniprotkb_tdt_AND_taxonomy_id_8782_2023_09_15.fasta', 'fasta'):
        if 'nucleotidylexotransferase' not in seq.description:
            continue
        if 'Fragment' in seq.description:
            continue
        if ' '.join(seq.description.split(' OX')[0].split('OS=')[1].split()[: 2]) in taxes:   
            continue
    
        opn_fasta.write(seq.format('fasta'))
        taxes.append(' '.join(seq.description.split(' OX')[0].split('OS=')[1].split()[: 2]))


In [ ]:
consensus_possitions = get_msa_info(f'/mnt/iscsidisk1/runs/runs-krivonos/PROJECTS/CONSENSUS_SEQ/TDT_bird/filtered.afa')
consensus = get_consensus(consensus_possitions)
print(consensus)

In [ ]:
opn_fasta = parse(f'/mnt/iscsidisk1/runs/runs-krivonos/PROJECTS/CONSENSUS_SEQ/test_balst.afa', 'fasta')
sequences = []

for seq in opn_fasta:
    
    sequences.append(seq.seq)
    print(seq.seq)

In [ ]:
for idx in range(len(sequences[0])):
    #print(sequences[0][idx], sequences[1][idx])
    if sequences[0][idx] != sequences[1][idx]:
        print(f'{sequences[0][idx]}{idx}{sequences[1][idx]}')

In [ ]:
interpro_tab = read_csv('bird_consensus_interpro_sacan.tsv', sep='\t', header=None)
pfam_data_bird = interpro_tab[interpro_tab[3] == 'Pfam'].sort_values(by=6)
bird_cons = 'MDRIRAPAVFSQRKRQKGMHSPNLSCSYEIKFNKFVIFIMQRKMGMTRRTFLMELARRKGFRVESELSDSVTHIVAENNSYLEVLDWLRGQAVGDSSRFELLDISWFTACMEAGRPVDSEMKYRLMEQDQSPPLNTPESEVPSFIASKVSQYSCQRKTTLNNYNKKFTDAFEIMAENYEFKENEIFCLEFLRAASVLKFLPFPVTRMKDIQGLPCMGDRVRDVIEEIIEEGESSRAKEVLNDERYKSFKQFTSVFGVGVKTSEKWYRMGLRTLEEVKADKTLKLSKMQKAGFLYYEDLVSCVSKAEADAVSLIVKNTVCTFLPDALVTITGGFRRGKKIGHDIDFLITNPGPREDDELLHKGLLLYCDIIESTFVKEQLPSRKVDAMDNFQKCFAILKLYQPRVDNSSYNTSKKFDMAEVKDWKAIRVDLVITPFEQYAYALLGWTGSRQFGRDLRRYATHERKMILDNHALYDRRKRIFLKAGSEEEIFAHLGLDYVEPWERNA'

In [ ]:
interpro_tab = read_csv('archea_termophil_interpro.tsv', sep='\t', header=None)
pfam_data_term = interpro_tab[interpro_tab[3] == 'Pfam'].sort_values(by=6)
tero_cons = 'MVNSQVASILEQIADLLELKGKSFFEARAYQRAARTLESLEEDVEDLYSRGALTSIPGIGKGIADKITEYVKTGTIGYLEDLKKEYPIGLIDLLRIPGLGPKKIKSLYKELGIKSIDELKKAAEEHRIRSLPGFGEKSEEKILKGIDLLESSSGRILLDVGYPYGRKIEEYLARSSGVDRVSLAGSLRRMKETIGDIDILASSEDPQKVMDAFISYPDVAEVLGKGDTKTSIILEDGVQVDLRVVDPSSFGAALQYFTGSKEHNIKLRKLAIDKGLKLNEYGLFRKDDKNIVAGKDEEDIYKHLGMDYIPPELREDRGEIEAALSHKLPRLVEPSDIKGDLHNHTDASDGVDSLEEMASAAQNKGYEYLGITDHSQSLKIANGLSDERLIEQIDEIRKLNEKWSDFRLLHGAECDILKDGSLDYSDEVLSQLDHVVASVHSWFKMDEEEMTERLIRAIENPKVTILGHPTGRIIGGREGYPVDMEAVIRAAGELNTALEINSSPERLDLDDELCKFAKEQGVPFSINTDAHNPNGLDDMRFGVGIARRGWLEKEDILNTLSLAELEKRLGL'

In [ ]:
for i in pfam_data_bird.index:
#, pfam_data_bird[7][i]-pfam_data_bird[6][i],
    print(pfam_data_bird[5][i], bird_cons[pfam_data_bird[6][i]:pfam_data_bird[7][i]])


In [ ]:
for i in pfam_data_term.index:
    #, pfam_data_term[7][i]-pfam_data_term[6][i]
    print(pfam_data_term[5][i], tero_cons[pfam_data_term[6][i]:pfam_data_term[7][i]])


In [ ]:
tero_cons[92:144]

In [ ]:
interpro_tab = read_csv('archea_termophil_interpro.tsv', sep='\t', header=None)

interpro_tab[interpro_tab[3] == 'Pfam']